In [9]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb
sys.path.append("../")
import src.soportecleaning as spcl

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [10]:
with open('../data/pickle/playlist.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 11)


,title,rank,date,artist,url,region,chart,trend,streams,week_in_charts,times_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,429,1


LastFM extraction

In [11]:
with open('../data/pickle/lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 24)


,url,artist,track,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,music_genre,gender,birthday_date,age
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,hip-hop,group,2013-06-15,9.000


Spotify extraction

In [12]:
with open('../data/pickle/spotify.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13992, 22)


,url,artist,track,popu,date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),76,2017-01-27,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [13]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          444929
left_only      13875
right_only         0
dtype: int64

In [15]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(458804, 51)


,title,rank,date_x,artist,url,region,chart,trend,streams,week_in_charts,times_in_charts,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,music_genre,gender,birthday_date,age,popu,date_y,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,429,1,migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,hip-hop,group,2013-06-15,9.000,76,2017-01-27,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [16]:
df.head(1)

,title,rank,date_x,artist,url,region,chart,trend,streams,week_in_charts,times_in_charts,artist_0,artist_1,artist_2,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,music_genre,gender,birthday_date,age,popu,date_y,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,429,1,migos,None,None,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,hip-hop,group,2013-06-15,9.000,76,2017-01-27,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [17]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [18]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [19]:
def highest_rank (url, row):
    return row[row['url'] == url].sort_values(by = 'rank').iloc[0,1]

In [20]:
def weeks_in_charts (url, df):
    return df[df['url'] == url].shape[0]

In [21]:
weeks_in_charts(df["url"], df)

458804

In [22]:
df[['url', 'chart']].value_counts()

url                                                    chart  
https://open.spotify.com/track/7sO5G9EABYOXQKNPNiE9NR  top200     3680
https://open.spotify.com/track/6gBFPUFcJLzWGx4lenP6h2  top200     1778
https://open.spotify.com/track/7m9OqQk4RVRkw9JJdeAw96  top200     1563
https://open.spotify.com/track/7GX5flRQZVHRAGd6B4TmDO  top200     1549
https://open.spotify.com/track/62vpWI1CHwFy7tMIcSStl8  top200     1474
                                                                  ... 
https://open.spotify.com/track/5Co8mnbtshK44Uh96aHxdu  top200        1
https://open.spotify.com/track/0yFWdhx0rDU4zNddqvfu33  top200        1
https://open.spotify.com/track/1xjYJJy5TFFVD9tStu2hfw  viral50       1
https://open.spotify.com/track/5Cevkgj9FyNRFIwY3xSCn8  top200        1
https://open.spotify.com/track/3ksNHUNCTP6Pan8rjFnmtw  top200        1
Length: 17277, dtype: int64

In [23]:
def highest_rank(lista_url):
    lista_test = []
    for url in lista_url:
        lista_test.append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    return lista_test

In [24]:
def highest_rank(dataframe, col):
    return df[df['url'] == link].sort_values(by = 'rank').iloc[0,1]

In [25]:
df.sort_values('rank').drop_duplicates('url')[['url', 'rank']]

,url,rank
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,1
356093,https://open.spotify.com/track/62aP9fBQKYKxi7P...,1
237889,https://open.spotify.com/track/2Yl4OmDby9iitgN...,1
443370,https://open.spotify.com/track/5elofKuRmgZmCVI...,1
237839,https://open.spotify.com/track/41cpvQ2GyGb2BRd...,1
...,...,...
37531,https://open.spotify.com/track/2dBLyJvf9T29rEi...,200
149834,https://open.spotify.com/track/5b1uZnebDRcR8Xu...,200
104198,https://open.spotify.com/track/6zmANU5l4qCHQrI...,200
406775,https://open.spotify.com/track/6myjRCWhMVgwgJE...,200


In [26]:
df.apply(lambda x: highest_rank(x, x['url']), axis = 1)

NameError: name 'link' is not defined

In [ ]:
df[df['url'] == lista_url[0]].nsmallest(1, 'rank').iloc[0,1]

1

In [ ]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].nsmallest(1, 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

  2%|▏         | 240/14650 [00:06<06:26, 37.30it/s]


KeyboardInterrupt: 

In [ ]:
def highest_ranking(url):
    return df[df['url'] == url].sort_values(by = 'rank').iloc[0,1]

In [ ]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

In [ ]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

In [ ]:
df.drop(['_merge'], axis = 1, inplace=True)

In [ ]:
df.head()

In [ ]:
df_export = df[['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend', 'streams', 'highest_rank', 'weeks_in_charts']]
df_export.head()

In [ ]:
df_artist = df_export[['artist', 'title', 'url']].drop_duplicates()
print(df_artist.shape)
df_artist.head()

In [ ]:
with open('../data/pickle/playlist.pickle', 'wb') as data_spotify:
    pickle.dump(df_export, data_spotify)

In [ ]:
with open('../data/pickle/artist.pickle', 'wb') as data_spotify:
    pickle.dump(df_artist, data_spotify)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [ ]:
df.head(1)

In [ ]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

In [ ]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

In [ ]:
dict_genres = dict(cuenta_generos.most_common(24))

In [ ]:
fuzz.ratio('hip-hop', "hip hop")

In [ ]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [ ]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [ ]:
df.sample(5)

In [ ]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [ ]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [ ]:
df.head(1)

In [ ]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [ ]:
df['gender'].value_counts() / df.shape[0]

Age from artist

-- taking the data from the summary

In [ ]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w{1,15} \d{1,2}, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

In [ ]:
df.sample()

df.dtypes

In [ ]:
df.dtypes

In [ ]:
df['playlist_date'] = pd.to_datetime(df['date'])
df['birthday_date'] = pd.to_datetime(df['birthday'])

In [ ]:
df['birthday_date'].value_counts()

In [ ]:
from datetime import date

def calculate_age(birthday):
    today = date.today()
    return today.year - birthday.year - ((today.month, today.day) < (birthday.month, birthday.day))

In [ ]:
df['age'] = df['birthday_date'].apply(calculate_age)
df['age'].value_counts()

In [ ]:
df['year'] = df['playlist_date'].dt.year
df['month'] = df['playlist_date'].dt.month

key and scale type mapping

In [ ]:
df['key_mapped'] = df['key'].replace(bb.dict_keys)
df['mode_mapped'] = df['mode'].replace(bb.dict_scale)

In [ ]:
df['key_mapped'].value_counts()

In [ ]:
df['mode_mapped'].value_counts()

# EDA

Total number of rows and columns of the dataframe

In [ ]:
df.shape

Counting duplicated rows

In [ ]:
df.duplicated().sum()

Counting null values

In [ ]:
df.isnull().sum() / df.shape[0]

In [ ]:
df.info()

Main statistic values for numeric columns

In [ ]:
df.describe().T

Main statistic values for categorical columns

In [ ]:
df.describe(include='object').T

# Questions / hypothesis

Firstly, both of the playlist will be splitted in different dataframes.

In [ ]:
top200 = df[df['chart'] == 'top200']
viral50 = df[df['chart'] == 'viral50']

## Most streamed artists

I will look up both of the list and see if top artist are the same in both of the lists. As the `streams` column is not available in the `viral50`, so we will use the `weeks_in_charts` column, that counts the number of occurences on the lists.

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

In [ ]:
top10_top200 = top200.groupby(['artist'])['weeks_in_charts'].sum().reset_index().sort_values(by = 'weeks_in_charts', ascending = False).head(5)
top10_top200

In [ ]:
top10_viral50 =viral50.groupby(['artist'])['weeks_in_charts'].sum().reset_index().sort_values(by = 'weeks_in_charts', ascending = False).head(5)
top10_viral50

As we see, both of them are different. We will see h the top artist evolve on the list on the given period. 

In [ ]:
artist_top200 = df[df['artist'].isin(top10_top200['artist'].tolist())].groupby(['artist', 'year', 'month'])['weeks_in_charts'].sum().reset_index().sort_values(by=['year', 'month'], ascending = True)
artist_top200['date'] = artist_top200['year'].astype(str) + '/' + artist_top200['month'].astype(str)
artist_top200.head()

In [ ]:
artist_viral50 = df[df['artist'].isin(top10_viral50['artist'].tolist())].groupby(['artist', 'year', 'month'])['weeks_in_charts'].sum().reset_index().sort_values(by=['year', 'month'], ascending = True)
artist_viral50['date'] = artist_viral50['year'].astype(str) + '/' + artist_viral50['month'].astype(str)
artist_viral50.head()

In [ ]:
#sns.set_theme(style="darkgrid")
#sns.set(rc={"figure.figsize":(25,10)})
fig, axs = plt.subplots(ncols=1, nrows = 2, figsize = (20, 10), sharex = True)
g1 = sns.lineplot(x="date", y="weeks_in_charts", hue="artist",  data=artist_top200, ax=axs[0])
g2 = sns.lineplot(x="date", y="weeks_in_charts", hue="artist",  data=artist_viral50 , ax=axs[1])
axs[0].set_title('top200')
axs[1].set_title('viral50')
plt.xticks(rotation = 90);

## Most streamed music genres

In [ ]:
df.head(1)

In [ ]:
top10_genre_top200 = top200.groupby(['clean_music_genre'])['weeks_in_charts'].sum().reset_index().sort_values(by = 'weeks_in_charts', ascending = False).head(11)
top10_genre_top200

In [ ]:
top10_genre_viral50 =viral50.groupby(['clean_music_genre'])['weeks_in_charts'].sum().reset_index().sort_values(by = 'weeks_in_charts', ascending = False).head(11)
top10_genre_viral50

Let's see the music genre distribution.

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows = 1, figsize = (10, 20), sharex = True)
axs0 = axs[0].pie(top10_genre_top200['weeks_in_charts'].tolist(), labels = top10_genre_top200['clean_music_genre'].tolist(), colors = sns.color_palette('muted'), autopct='%.0f%%')
axs1 = axs[1].pie(top10_genre_viral50['weeks_in_charts'].tolist(), labels = top10_genre_viral50['clean_music_genre'].tolist(), colors = sns.color_palette('muted'), autopct='%.0f%%')
plt.show()

In [ ]:
genre_top200 = df.groupby(['clean_music_genre', 'year', 'month'])['weeks_in_charts'].sum().reset_index().sort_values(by=['year', 'month'], ascending = True)
genre_top200['date'] = genre_top200['year'].astype(str) + '/' + genre_top200['month'].astype(str)
genre_top200.head()

In [ ]:
artist_viral50 = df[df['artist'].isin(top10_viral50['artist'].tolist())].groupby(['artist', 'year', 'month'])['weeks_in_charts'].sum().reset_index().sort_values(by=['year', 'month'], ascending = True)
artist_viral50['date'] = artist_viral50['year'].astype(str) + '/' + artist_viral50['month'].astype(str)
artist_viral50.head()

## Gender distibution

Let's see how genders  are distributed in both of the playlist.

In [ ]:
top200.groupby(['gender'])['weeks_in_charts'].count().reset_index()

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows = 1, figsize = (10, 20), sharex = True)
axs0 = axs[0].pie(top200.groupby(['gender'])['weeks_in_charts'].count().reset_index()['weeks_in_charts'].tolist(), 
    labels = top200.groupby(['gender'])['weeks_in_charts'].count().reset_index()['gender'].tolist(), colors = sns.color_palette('muted'), autopct='%.0f%%')
axs1 = axs[1].pie(viral50.groupby(['gender'])['weeks_in_charts'].count().reset_index()['weeks_in_charts'].tolist(), 
    labels = viral50.groupby(['gender'])['weeks_in_charts'].count().reset_index()['gender'].tolist(), colors = sns.color_palette('muted'), autopct='%.0f%%')
plt.show()

Let's see the distribution of gender during the years.

In [ ]:
gender_top200 = top200.groupby(['gender', 'year', 'month'])['weeks_in_charts'].sum().reset_index().sort_values(by=['year', 'month'], ascending = True)
gender_top200['date'] = gender_top200['year'].astype(str) + '/' + gender_top200['month'].astype(str)
gender_top200.head()

In [ ]:
gender_viral50 = viral50.groupby(['gender', 'year', 'month'])['weeks_in_charts'].sum().reset_index().sort_values(by=['year', 'month'], ascending = True)
gender_viral50['date'] = gender_viral50['year'].astype(str) + '/' + gender_viral50['month'].astype(str)
gender_viral50.head()

In [ ]:
#sns.set_theme(style="darkgrid")
#sns.set(rc={"figure.figsize":(25,10)})
fig, axs = plt.subplots(ncols=1, nrows = 2, figsize = (20, 10), sharex = True)
g1 = sns.lineplot(x="date", y="weeks_in_charts", hue="gender",  data=gender_top200, ax=axs[0])
g2 = sns.lineplot(x="date", y="weeks_in_charts", hue="gender",  data=gender_viral50 , ax=axs[1])
axs[0].set_title('top200')
axs[1].set_title('viral50')
axs[0].axvline('2020/1', 0, max(gender_viral50['weeks_in_charts']), linestyle='dashed')
axs[1].axvline('2020/1', 0, max(gender_viral50['weeks_in_charts']), linestyle='dashed')
plt.xticks(rotation = 90);

In the `top200` there seems to be no advanced. The music displayed by this playlist is mostly male dominated. However, in the `viral50` playlist there seems to be hope for females. 

## Genres and genders

Let's see if there is a relation between the genres and the genders of the artist.

## How was the music during covid

So my hypothesis is that during covid, there was a change of tendencia in how people listened to music. Maybe there was an uplifting mood that they look up for or sinking in in sandness -- lets see the tendencies.

## AKK

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)